In [1]:
import numpy as np
import pandas as pd
import tensorflow_recommenders as tfrs
import tensorflow as tf
from typing import Dict, Text

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
import pathlib
import os

user_file = "\\Data\\Combined Data.csv"
caregiver_file = "\\Data\\Processed Caregiver Data.csv"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_dataframe = pd.read_csv(str(parent_path) + user_file)
caregiver_dataframe = pd.read_csv(str(parent_path) + caregiver_file)



In [3]:
unique_masalah_user = np.unique(' '.join(user_dataframe['Tipe_Masalah']).split(' '))
unique_masalah_caregiver = np.unique(' '.join(user_dataframe['Caregiver_Tipe_Masalah']).split(' '))
np.delete(unique_masalah_caregiver, np.where(unique_masalah_caregiver == ''))
unique_masalah_caregiver = np.delete(unique_masalah_caregiver, 0)

unique_masalah_caregiver = unique_masalah_caregiver.tolist()

for index, col in enumerate(unique_masalah_caregiver):
        unique_masalah_caregiver[index] = 'Caregiver-'+col


In [4]:
def convert_categorical_data(df, col='Tipe_Masalah'):
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(col=='Tipe_Masalah'):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [5]:
convert_categorical_data(user_dataframe)
convert_categorical_data(user_dataframe, col='Caregiver_Tipe_Masalah')
convert_categorical_data(caregiver_dataframe, col='Caregiver_Tipe_Masalah')


user_dataframe.drop(["USER_ID"], 1,inplace=True)
user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15916\1340759235.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["USER_ID"], 1,inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15916\1340759235.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)


In [6]:

user_dataframe.head()



,Gender,Age,Caregiver_Gender,Caregiver_Age,ADHD-Hiperaktif-dan-kurang-fokus,Depresi,Gangguan-kecemasan,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,Pria,33,Pria,19,1,0,0,0,0,0,1,0,0,0,0,0
1,Pria,33,Pria,35,1,0,0,0,0,0,1,0,1,0,1,0
2,Pria,29,Perempuan,33,0,0,0,1,1,0,0,0,0,1,1,0
3,Pria,29,Pria,38,0,0,0,1,1,0,1,0,0,1,0,0
4,Perempuan,32,Pria,35,1,0,1,0,0,0,1,0,1,0,1,0


In [7]:

caregiver_dataframe.head()

,CAREGIVER_ID,Caregiver_Gender,Caregiver_Age,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,1,Perempuan,28,1,0,0,1,1,0
1,2,Pria,33,0,1,0,0,0,1
2,3,Pria,19,1,0,0,0,0,0
3,4,Perempuan,18,0,0,0,1,0,1
4,5,Perempuan,36,0,0,1,0,0,1


In [8]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [9]:
batch_size = 5# A small batch sized is used for demonstration purposes
caregiver_ds = df_to_dataset(caregiver_dataframe, batch_size=batch_size,  shuffle=False)
user_ds = df_to_dataset(user_dataframe, batch_size=batch_size)

In [10]:
feature_columns = []

number_feature = ["Age"]
number_feature += unique_masalah_user.tolist()
print(unique_masalah_caregiver)
# numeric cols
for header in number_feature:
    feature_columns.append(feature_column.numeric_column(header))

age_feature = ["Age"]
for col in age_feature:
    age = feature_column.numeric_column(col)
    age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
    feature_columns.append(age_buckets)

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [11]:
# indicator_columns
indicator_column_names = ['Gender']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, user_dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)



In [12]:
for col in feature_columns:
    print(col)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='ADHD-Hiperaktif-dan-kurang-fokus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Depresi', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-kecemasan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-makan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Gangguan-stres-pascatrauma', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Skizofrenia', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(17, 21, 25, 29, 33, 37, 41, 46))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='G

In [13]:
feature_layer_user= tf.keras.layers.DenseFeatures(feature_columns)

In [14]:
feature_columns = []

number_feature = ["Caregiver_Age"]
number_feature += unique_masalah_caregiver
print(unique_masalah_caregiver)
# numeric cols

for header in number_feature:
    feature_columns.append(feature_column.numeric_column(header))

age_feature = ["Caregiver_Age"]
for col in age_feature:
    age = feature_column.numeric_column(col)
    age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
    feature_columns.append(age_buckets)

col_name = 'Caregiver_Gender'
categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, user_dataframe[col_name].unique())
indicator_column = feature_column.indicator_column(categorical_column)
feature_columns.append(indicator_column)

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [15]:
feature_layer_caregiver= tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
class RecomendModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    embedding_dimension = 32

    # Set up a model for representing users.
    self.user_model = tf.keras.Sequential([
        feature_layer_user,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a model for representing caregiver.
    self.caregiver_model = tf.keras.Sequential([
        feature_layer_caregiver,
        layers.Dense(64),
        layers.Dense(32)
    ])

    # Set up a task to optimize the model and compute metrics.
    self.task = tfrs.tasks.Retrieval(
      metrics=tfrs.metrics.FactorizedTopK(
        candidates=caregiver_ds.batch(5).map(self.caregiver_model)
      )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    
    caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
    user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    
    
    # We pick out the user features and pass them into the user model.
    
    user_embeddings = self.user_model({
        'Age':features['Age'],
        'Gender':features['Gender'],
        'ADHD-Hiperaktif-dan-kurang-fokus':features['ADHD-Hiperaktif-dan-kurang-fokus'],
        'Depresi':features['Depresi'],
        'Gangguan-kecemasan':features['Gangguan-kecemasan'],
        'Gangguan-makan':features['Gangguan-makan'],
        'Gangguan-stres-pascatrauma':features['Gangguan-stres-pascatrauma'],
        'Skizofrenia':features['Skizofrenia']
    })
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.caregiver_model({
        'Caregiver_Age':features['Caregiver_Age'],
        'Caregiver_Gender':features['Caregiver_Gender'],
        'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus':features['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'],
        'Caregiver-Depresi':features['Caregiver-Depresi'],
        'Caregiver-Gangguan-kecemasan':features['Caregiver-Gangguan-kecemasan'],
        'Caregiver-Gangguan-makan':features['Caregiver-Gangguan-makan'],
        'Caregiver-Gangguan-stres-pascatrauma':features['Caregiver-Gangguan-stres-pascatrauma'],
        'Caregiver-Skizofrenia':features['Caregiver-Skizofrenia']
        
    })
    

    # The task computes the loss and the metrics.

    return self.task(user_embeddings, positive_movie_embeddings, compute_metrics=not training)

In [17]:
caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']

In [18]:

for feature_batch in caregiver_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'])

Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
A batch of ages: tf.Tensor([1 0 1 0 0], shape=(5,), dtype=int32)


In [29]:
import tensorflow_datasets as tfds

c:\users\asus\anaconda3\envs\notebook\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [34]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(user_ds, 200, train_split=0.8, val_split=0.2, test_split=0,)

In [37]:
model = RecomendModel()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/50
AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB
AAAAAAAAAAAAAAAAAA
BBBBBBBBBBBBBB
40/40 [==============================] - 1s 10ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 55.9835 - regularization_loss: 0.0000e+00 - total_loss: 55.9835
Epoch 2/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 19.1213 - regularization_loss: 0.0000e+00 - total_loss: 19.1213
Epoch 3/50
40/40 [==============================] - 0s 9ms/step - facto

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 10.3431 - regularization_loss: 0.0000e+00 - total_loss: 10.3431
Epoch 11/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 12.5725 - regularization_loss: 0.0000e+00 - total_loss: 12.5725
Epoch 12/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_ca

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 7.1830 - regularization_loss: 0.0000e+00 - total_loss: 7.1830
Epoch 30/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.8100 - regularization_loss: 0.0000e+00 - total_loss: 6.8100
Epoch 31/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_catego

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.6246 - regularization_loss: 0.0000e+00 - total_loss: 6.6246
Epoch 49/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.4869 - regularization_loss: 0.0000e+00 - total_loss: 6.4869
Epoch 50/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_catego

In [26]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [20]:
for feature_batch in caregiver_ds:
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-s

In [21]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
brute_force.index_from_dataset(  
       caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], model.caregiver_model(features)))
)





In [22]:
a =['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el,([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test)

{'Gender': ['Pria'], 'Age': [30], 'ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Depresi': [1], 'Gangguan-kecemasan': [0], 'Gangguan-makan': [0], 'Gangguan-stres-pascatrauma': [1], 'Skizofrenia': [0]}


In [23]:
a =['Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan','Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test_Caregiver = dict((el,([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test_Caregiver)

{'Caregiver_Gender': ['Pria'], 'Caregiver_Age': [30], 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Caregiver-Depresi': [1], 'Caregiver-Gangguan-kecemasan': [0], 'Caregiver-Gangguan-makan': [0], 'Caregiver-Gangguan-stres-pascatrauma': [1], 'Caregiver-Skizofrenia': [0]}


In [24]:
# Get predictions for user 42.
a, titles = brute_force(dictionary_Test, k=2)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [11 24]


In [25]:
tensor = model.caregiver_model(dictionary_Test_Caregiver)
tensor.numpy()

array([[ 0.05826065, -0.38476425, -0.40455246,  0.15347545, -0.9849558 ,
         0.6024401 ,  0.87596047,  1.9819466 , -0.11216472,  0.64404947,
        -0.7053842 , -0.3897458 ,  1.7571347 ,  0.18089873,  0.37688297,
        -0.23129736,  0.15130869,  0.7019512 , -0.7580117 ,  0.58990633,
         0.29467872, -0.7741382 , -0.5695449 ,  0.23612425, -0.42787105,
        -0.31546247, -1.6128621 , -1.0114098 , -1.0022382 ,  2.004491  ,
        -0.13656217, -0.55576605]], dtype=float32)

In [26]:
tensor = model.user_model(dictionary_Test)
tensor.numpy()

array([[ 0.40695393, -0.75670004,  0.44062   ,  0.03337876,  0.3510351 ,
        -0.8266126 ,  0.47377875, -0.00593777,  0.28147358,  0.50841457,
         0.25378108,  0.0127746 , -0.23315774,  0.57836205, -0.45241782,
        -0.4480724 , -0.38965246,  1.5780258 , -1.0968956 , -0.15245171,
        -0.29448467,  0.30920178,  0.26669517, -0.49621144,  0.512001  ,
        -1.1214398 , -0.95028883,  0.18500848, -0.31177223,  1.1428536 ,
        -0.35230073,  0.325258  ]], dtype=float32)

In [27]:
processed_ds = caregiver_ds.batch(5).map(model.caregiver_model)


In [28]:
model.save

<bound method Model.save of <__main__.RecomendModel object at 0x000001E21200E910>>

In [30]:
processed_ds

<MapDataset element_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None)>

In [29]:
model.user_model.save('saved_model/user_query_v1')
model.caregiver_model.save('saved_model/caregiver_query_v1')

INFO:tensorflow:Assets written to: saved_model/user_query_v1\assets


INFO:tensorflow:Assets written to: saved_model/user_query_v1\assets


INFO:tensorflow:Assets written to: saved_model/caregiver_query_v1\assets


INFO:tensorflow:Assets written to: saved_model/caregiver_query_v1\assets


In [34]:

MAIN_USER_MODEL =  tf.saved_model.load('saved_model/user_query_v1')
MAIN_CAREGIVER_MODEL = tf.saved_model.load('saved_model/caregiver_query_v1')

In [ ]:
saved_test = caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))

In [ ]:
MAIN_USER_MODEL(dictionary_Test)

In [ ]:
MAIN_CAREGIVER_MODEL(dictionary_Test_Caregiver)

In [ ]:
print("s")

In [33]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model_v1/user_query_v1/saved_model.pb') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('saved_model_tflite_v1/text_query_v1', 'wb') as f:
  f.write(tflite_model)

OSError: SavedModel file does not exist at: saved_model_v1/user_query_v1/saved_model.pb\{saved_model.pbtxt|saved_model.pb}